# Test fonction is_allowed_node

In [1]:
from clang import cindex

import os

ALLOWED_PATHS = ['test_code/Includes/test_code.h', 'test_code/src/test_code.cpp']

import os

def is_allowed_node(node, ALLOWED_PATHS):
    """
    Retourne True si le nœud appartient à l'un des fichiers autorisés.
    Si le nœud n'a pas de localisation (par exemple la racine), on le considère autorisé.
    Pour plus de flexibilité, si le basename (nom du fichier) du nœud correspond à celui
    d'un chemin autorisé, on accepte également.
    """
    if node.location and node.location.file:
        # Normalisation du chemin absolu du fichier du nœud
        file_path = os.path.normpath(os.path.abspath(node.location.file.name))
        for allowed in ALLOWED_PATHS:
            # Normalisation du chemin absolu autorisé
            allowed_abs = os.path.normpath(os.path.abspath(allowed))
            # Option stricte : le chemin complet correspond ou est dans le même répertoire
            if file_path == allowed_abs or file_path.startswith(os.path.dirname(allowed_abs)):
                return True
            # Option plus flexible : le nom du fichier correspond (pour gérer les cas src/../Include)
            if os.path.basename(file_path) == os.path.basename(allowed_abs):
                return True
        return False
    return True

def print_ast(node, indent=0, output_file=None):
    indent_str = " " * indent

    # Récupération de la localisation
    loc = node.location
    file_name = loc.file.name if loc and loc.file else "None"
    line = loc.line if loc and loc.line != 0 else "?"
    column = loc.column if loc and loc.column != 0 else "?"

    # Récupération du type (si disponible)
    try:
        type_spelling = node.type.spelling
    except Exception:
        type_spelling = "None"

    # Autres informations
    displayname = node.displayname if node.displayname else ""
    is_def = node.is_definition()
    access = node.access_specifier
    raw_comment = node.raw_comment if node.raw_comment else ""

    # Récupération de l'étendue (extent)
    extent = node.extent
    if extent:
        start = extent.start
        end = extent.end
        extent_str = f"{start.file.name if start.file else 'None'}({start.line},{start.column}) -> {end.file.name if end.file else 'None'}({end.line},{end.column})"
    else:
        extent_str = "None"

    # Création d'une ligne résumant toutes les informations du nœud
    line_out = (
        f"{indent_str}{node.kind} |Is allowed {is_allowed_node(node,ALLOWED_PATHS)}| Spelling: '{node.spelling}' | "
        f"Displayname: '{displayname}' | Location: {file_name} (line: {line}, col: {column}) | "
        f"IsDefinition: {is_def} | Type: {type_spelling} | Access: {access} | "
        f"Extent: {extent_str} | RawComment: '{raw_comment}'"
    )

    # Écriture dans le fichier ou affichage sur la console
    if output_file:
        output_file.write(line_out + "\n")
    else:
        print(line_out)

    # Parcours récursif pour chaque enfant
    for child in node.get_children():
        print_ast(child, indent + 2, output_file)

def main():
    # Création de l'index
    index = cindex.Index.create()

    # Spécification du fichier source et des options de compilation
    source_file = "test_code/src/test_code.cpp"
    args = ["-I./test_code/Include", "-std=c++11"]

    # Analyse du fichier pour obtenir le TranslationUnit (AST)
    tu = index.parse(source_file, args=args)

    # Affichage des diagnostics le cas échéant
    if tu.diagnostics:
        print("Diagnostics:")
        for diag in tu.diagnostics:
            print(diag)

    # Ouverture du fichier de sortie en écriture
    with open("exp_results/ast_output.txt", "w", encoding="utf-8") as f:
        f.write("AST for: " + source_file + "\n")
        print_ast(tu.cursor, output_file=f)

    print("AST sauvegardé dans ast_output.txt")

if __name__ == '__main__':
    main()


Diagnostics:
/usr/include/wchar.h:35:10: fatal error: 'stddef.h' file not found
AST sauvegardé dans ast_output.txt


# Normalisation des chemins 

In [2]:
import os

path = "test_code/src/../Include/test_code.h"
normalized_path = os.path.normpath(path)
print(normalized_path)  # Affiche: test_code/Include/test_code.h

test_code/Include/test_code.h
